DOWNLOAD YOUTUBE VIDEO

In [5]:
import yt_dlp

# Enter the YouTube video URL
url = "https://www.youtube.com/watch?v=NaDfRqFwKVM&ab_channel=NBCNews"

ydl_opts = {
    "outtmpl": "videos/%(title)s.%(ext)s",  # save as video title
    "format": "best"
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])


[youtube] Extracting URL: https://www.youtube.com/watch?v=NaDfRqFwKVM&ab_channel=NBCNews
[youtube] NaDfRqFwKVM: Downloading webpage
[youtube] NaDfRqFwKVM: Downloading tv simply player API JSON
[youtube] NaDfRqFwKVM: Downloading tv client config
[youtube] NaDfRqFwKVM: Downloading tv player API JSON
[info] NaDfRqFwKVM: Downloading 1 format(s): 18
[download] Sleeping 3.00 seconds as required by the site...
[download] Destination: videos\Michael Phelps reflects on depression and mental health： 'I saw it as a sign of weakness'.mp4
[download] 100% of    4.83MiB in 00:00:00 at 7.06MiB/s   


DOWNLOAD AUDIO FROM YOUTUBE VIDEO

In [6]:
import yt_dlp

url = "https://www.youtube.com/watch?v=NaDfRqFwKVM&ab_channel=NBCNews"

ydl_opts = {
    "format": "bestaudio/best",
    "outtmpl": "Audios/%(title)s.%(ext)s",
    "postprocessors": [
        {
            "key": "FFmpegExtractAudio",
            "preferredcodec": "mp3",   # or "wav"
            "preferredquality": "192", # kbps
        }
    ],
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

print("✅ Audio extracted directly as MP3!")


[youtube] Extracting URL: https://www.youtube.com/watch?v=NaDfRqFwKVM&ab_channel=NBCNews
[youtube] NaDfRqFwKVM: Downloading webpage
[youtube] NaDfRqFwKVM: Downloading tv simply player API JSON
[youtube] NaDfRqFwKVM: Downloading tv client config
[youtube] NaDfRqFwKVM: Downloading tv player API JSON
[info] NaDfRqFwKVM: Downloading 1 format(s): 251
[download] Sleeping 3.00 seconds as required by the site...
[download] Destination: Audios\Michael Phelps reflects on depression and mental health： 'I saw it as a sign of weakness'.webm
[download] 100% of    1.20MiB in 00:00:00 at 3.22MiB/s   
[ExtractAudio] Destination: Audios\Michael Phelps reflects on depression and mental health： 'I saw it as a sign of weakness'.mp3
Deleting original file Audios\Michael Phelps reflects on depression and mental health： 'I saw it as a sign of weakness'.webm (pass -k to keep)
✅ Audio extracted directly as MP3!


EXTRACT AUDIO FROM LOCAL VIDEO

In [9]:
from moviepy import VideoFileClip

# Path to downloaded video
video_path = "videos/Michael Phelps reflects on depression and mental health： 'I saw it as a sign of weakness'.mp4"
output_audio_path = "Audios/sample_audio.mp3"

# Load video and extract audio
video = VideoFileClip(video_path)
video.audio.write_audiofile(output_audio_path)

print("✅ Audio extracted as MP3!")


MoviePy - Writing audio in Audios/sample_audio.mp3


MoviePy - Done.
✅ Audio extracted as MP3!


GENERATE EMBEDDINGS

In [11]:
from torchvision import models, transforms
from PIL import Image
import torch
import cv2

# Load pre-trained model (e.g., ResNet)
model = models.resnet50(pretrained=True)
model.eval()

# Video path
video_path = "videos/Michael Phelps reflects on depression and mental health： 'I saw it as a sign of weakness'.mp4"
cap = cv2.VideoCapture(video_path)

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

embeddings = []

while True:
    ret, frame = cap.read()
    if not ret:
        break
    img = transform(frame).unsqueeze(0)
    with torch.no_grad():
        emb = model(img)
    embeddings.append(emb)

cap.release()
print(f"Created {len(embeddings)} frame embeddings")


Created 2821 frame embeddings


In [13]:
import numpy as np
np.array(embeddings).shape

(2821, 1, 1000)

In [ ]:
from transformers import ViTFeatureExtractor, ViTModel
from PIL import Image
import torch

image = Image.open("frame.jpg")
extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")
model = ViTModel.from_pretrained("google/vit-base-patch16-224")

inputs = extractor(images=image, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
embedding = outputs.last_hidden_state.mean(dim=1)  # pooled embedding


LIMB POSITION

In [15]:
from ultralytics import YOLO

model = YOLO("yolov8n-pose.pt")
results = model("videos/Michael Phelps reflects on depression and mental health： 'I saw it as a sign of weakness'.mp4")

for frame in results:
    keypoints = frame.keypoints  # x, y, confidence



WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/2821) c:\Users\Tathagata\Desktop\Code\deepheal\videos\Michael Phelps reflects on depression and mental health 'I saw it as a sign of weakness'.mp4: 384x640 2 persons, 85.8ms
video 1/1 (frame 2/2821) c:\Users\Tathagata\Desktop\Code\deepheal\videos\Michael Phelps reflects on depression and mental health 'I saw it as a sign of weakness'.mp4: 384x640 2 persons, 20.3ms
video 1/1 (frame 3/2821) c:\Users\Tathagata\Desktop\Code\deepheal\videos\

In [ ]:
from ultralytics import YOLO
import cv2

# Load YOLO pose model
model = YOLO("yolov8n-pose.pt")

# Open video file
cap = cv2.VideoCapture("video.mp4")
fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define output writer
out = cv2.VideoWriter("output_pose.mp4",
                      cv2.VideoWriter_fourcc(*"mp4v"),
                      fps, (w, h))

while True:
    ret, frame = cap.read()
    if not ret:
        break  # end of video

    # Run YOLO inference on the frame
    results = model(frame, verbose=False)

    # Draw skeleton on frame
    annotated_frame = results[0].plot()

    # Save to output video
    out.write(annotated_frame)

    # (Optional) Show live preview
    cv2.imshow("Pose Detection", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print("✅ Pose video saved as output_pose.mp4")



video 1/1 (frame 1/2821) c:\Users\Tathagata\Desktop\Code\deepheal\videos\Michael Phelps reflects on depression and mental health 'I saw it as a sign of weakness'.mp4: 384x640 2 persons, 38.3ms
video 1/1 (frame 2/2821) c:\Users\Tathagata\Desktop\Code\deepheal\videos\Michael Phelps reflects on depression and mental health 'I saw it as a sign of weakness'.mp4: 384x640 2 persons, 19.2ms
video 1/1 (frame 3/2821) c:\Users\Tathagata\Desktop\Code\deepheal\videos\Michael Phelps reflects on depression and mental health 'I saw it as a sign of weakness'.mp4: 384x640 3 persons, 14.7ms
video 1/1 (frame 4/2821) c:\Users\Tathagata\Desktop\Code\deepheal\videos\Michael Phelps reflects on depression and mental health 'I saw it as a sign of weakness'.mp4: 384x640 2 persons, 20.7ms
video 1/1 (frame 5/2821) c:\Users\Tathagata\Desktop\Code\deepheal\videos\Michael Phelps reflects on depression and mental health 'I saw it as a sign of weakness'.mp4: 384x640 2 persons, 15.1ms
video 1/1 (frame 6/2821) c:\Users\